**Name: Venkata Sai Pallavi Pallapolu**

**Assignment Number: Week 09 Machine Learning with Scikit-learn**

## Q1: Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

**Importing the Machine_Learning_Review_Classification file**

In [183]:
%run Machine_Learning_Review_Classification.ipynb

3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]
Pandas version: 2.2.3
Matplotlib version: 3.8.4
Numpy version: 1.26.4
SciKitLearn version: 1.4.2
My working directory:
C:\Users\pvspa
My new working directory:
C:\Users\pvspa
              precision    recall  f1-score   support

         0.0       0.65      1.00      0.79       970
         1.0       0.00      0.00      0.00       530

    accuracy                           0.65      1500
   macro avg       0.32      0.50      0.39      1500
weighted avg       0.42      0.65      0.51      1500


Model                  Train    Test
-------------------------------------------
Logistic               0.6467   0.608 

Model                  Train    Test
-------------------------------------------
Logistic               0.6467   0.608 
Null                   0.6467   0.608 
              precision    recall  f1-score   support

         0.0       0.76      0.85      0.80       970
         1.0

In [184]:
#recalling the final results of the file
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.6467   0.608 
Null                   0.6467   0.608 
Logistic_L1_C_1        0.732    0.716 
Logistic_L1_C_01       0.726    0.706 
Logistic_L1_C_10       0.7347   0.718 
Logistic_L1_C_auto     0.7233   0.708 
Logistic_SL1_C_auto    0.7307   0.714 
RandomForest_noCV      0.9993   0.684 
RandomForest_CV        0.9993   0.706 
RandomForest_CV2       0.7267   0.7   


**Interpretation:** Based on the results from the Machine_Learning_Review_Classification file, among the models with and without regularization and random forest, Logistic_L1_C_10 model is found to have the best overall performance with 73.47% training accuracy and 71.8% test accuracy. Also, another model that is Logistic_L1_C_1 has attained the near similar accuracies on both the training and test data. C here represents the regularization strength, the smaller it is, the larger the regularization strength. Logistic_L1_C_10 and Logistic_L1_C_1 models have achieved the highest generalization accuracy on the holdout(test) set, while also maintaining strong performance on the training data without overfitting, unlike the RandomForest models with and without cross validation, which had very high training accuracy (0.9993) but lower test accuracy (0.684, 0.706), an indication of overfitting. Overall, Logistic_L1_C_10 model has the best performance because it strikes the ideal balance between high accuracy and generalization, achieving the highest test performance without signs of overfitting. Its training accuracy (0.7347) is high but not excessively so, indicating that the model learned well from the training data without overfitting. It achieved the highest test accuracy (0.718), which means it generalizes best to unseen data. 

## Q2: Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

**preprocessing**

In [185]:
# Create mortality variable
df_patient['mortality'] = np.where(df_patient['DateOfDeath'].isnull(), 0,1)
# Examine
df_patient['mortality']

0        0
1        1
2        0
3        1
4        0
        ..
19995    0
19996    0
19997    0
19998    0
19999    0
Name: mortality, Length: 20000, dtype: int32

In [186]:
#convert into gender and race variable values to categorical
df_patient["Gender"] = df_patient["Gender"].map({'female': 0, 'male': 1})
df_patient["Race"] = df_patient["Race"].map({'hispanic': 0,
    'white': 1,
    'black': 2,
    'other': 3})

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath,mortality,Age_years
0,1,1962-02-27,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2013-04-27,2018-06-01,NaN,0,52.843258
1,2,1959-08-18,1,1,0,0,0,0,0,0,...,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02,1,55.373032
2,3,1946-02-15,0,1,0,0,0,0,0,0,...,0,0,1,0,0,2011-11-05,2015-11-13,NaN,0,68.876112
3,4,1979-07-27,0,1,0,0,0,0,0,1,...,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17,1,35.433265
4,5,1983-02-19,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2006-09-22,2018-06-01,NaN,0,31.865845


In [187]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)
df_patient['Age_years'].describe()

count    20000.000000
mean        47.247474
std         18.145086
min         15.753593
25%         31.733744
50%         47.099247
75%         62.924025
max         78.743326
Name: Age_years, dtype: float64

In [188]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth','Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)

In [189]:
#defining the data for predictors and outcome
X = df_patient[list(vars_left)]
y = df_patient["mortality"]

In [190]:
X.shape
y.shape

(20000, 25)

(20000,)

In [191]:
## Split Data into training and sample
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y),test_size=0.20,random_state=42)

In [192]:
X_train.shape

(16000, 25)

In [193]:
X_test.shape

(4000, 25)

In [194]:
y_train.shape

(16000,)

In [195]:
y_test.shape

(4000,)

**Logistic Model using lbfgs Solver**

In [196]:
import time
from sklearn import linear_model
## Define model parameters
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver = 'lbfgs',
                                      penalty = None, # excluding the penalty
                                     max_iter = 5000) # making the iterations to 5000
## fit model using data with .fit
start_time = time.time()
clf.fit(X_train,y_train)
end_time = time.time()
elapsed_time = end_time - start_time

LogisticRegression(max_iter=5000, penalty=None)

In [197]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['lbfgs'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)), elapsed_time)

In [198]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:30} {1:25} {2:25} {3:15}".format('Solver used', 'Training subset accuracy', 'Holdout subset accuracy', 'Time taken'))
    print('-' * 100)
    for i in x1.keys():
        print("{0:30} {1:<25.4f} {2:<25.4f} {3:<15.4f}".format(i, x1[i][0], x1[i][1], x1[i][2]))

In [199]:
get_results(result_scores)


Solver used                    Training subset accuracy  Holdout subset accuracy   Time taken     
----------------------------------------------------------------------------------------------------
lbfgs                          0.7485                    0.7358                    0.4671         


**Logistic model using newton-cg Solver**

In [200]:
## Define model parameters
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver = 'newton-cg',
                                      penalty = None, # excluding the penalty
                                     max_iter = 5000) # making the iterations to 5000
## fit model using data with .fit
start_time = time.time()
clf.fit(X_train,y_train)
end_time = time.time()
elapsed_time = end_time - start_time

LogisticRegression(max_iter=5000, penalty=None, solver='newton-cg')

In [201]:
## Score the Model on Training and Testing Set
result_scores['newton-cg'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)), elapsed_time)
get_results(result_scores)


Solver used                    Training subset accuracy  Holdout subset accuracy   Time taken     
----------------------------------------------------------------------------------------------------
lbfgs                          0.7485                    0.7358                    0.4671         
newton-cg                      0.7488                    0.7362                    0.1401         


**Logistic model using newton-cholesky Solver**

In [202]:
## Define model parameters
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver = 'newton-cholesky',
                                      penalty = None, # excluding the penalty
                                     max_iter = 5000) # making the iterations to 5000
## fit model using data with .fit
start_time = time.time()
clf.fit(X_train,y_train)
end_time = time.time()
elapsed_time = end_time - start_time

LogisticRegression(max_iter=5000, penalty=None, solver='newton-cholesky')

In [203]:
## Score the Model on Training and Testing Set
result_scores['newton-cholesky'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)), elapsed_time)
get_results(result_scores)


Solver used                    Training subset accuracy  Holdout subset accuracy   Time taken     
----------------------------------------------------------------------------------------------------
lbfgs                          0.7485                    0.7358                    0.4671         
newton-cg                      0.7488                    0.7362                    0.1401         
newton-cholesky                0.7488                    0.7362                    0.0781         


**Logistic model using sag Solver**

In [204]:
## Define model parameters
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver = 'sag',
                                      penalty = None,
                                     max_iter = 5000) # making the iterations to 5000
## fit model using data with .fit
start_time = time.time()
clf.fit(X_train,y_train)
end_time = time.time()
elapsed_time = end_time - start_time

LogisticRegression(max_iter=5000, penalty=None, solver='sag')

In [205]:
## Score the Model on Training and Testing Set
result_scores['sag'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)), elapsed_time)
get_results(result_scores)


Solver used                    Training subset accuracy  Holdout subset accuracy   Time taken     
----------------------------------------------------------------------------------------------------
lbfgs                          0.7485                    0.7358                    0.4671         
newton-cg                      0.7488                    0.7362                    0.1401         
newton-cholesky                0.7488                    0.7362                    0.0781         
sag                            0.7488                    0.7362                    2.0829         


**Logistic model using saga Solver**

In [206]:

clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver = 'saga',
                                      penalty = None,
                                     max_iter = 5000) # could change to lbfgs!
## fit model using data with .fit
start_time = time.time()
clf.fit(X_train,y_train)
end_time = time.time()
elapsed_time = end_time - start_time

LogisticRegression(max_iter=5000, penalty=None, solver='saga')

In [207]:
## Score the Model on Training and Testing Set
result_scores['saga'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)), elapsed_time)
get_results(result_scores)


Solver used                    Training subset accuracy  Holdout subset accuracy   Time taken     
----------------------------------------------------------------------------------------------------
lbfgs                          0.7485                    0.7358                    0.4671         
newton-cg                      0.7488                    0.7362                    0.1401         
newton-cholesky                0.7488                    0.7362                    0.0781         
sag                            0.7488                    0.7362                    2.0829         
saga                           0.7488                    0.7362                    3.4706         


## Q3: Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:

## Solver used
	

## Training subset accuracy
	

## Holdout subset accuracy
	

## Time taken

In [208]:
#final results 
get_results(result_scores)


Solver used                    Training subset accuracy  Holdout subset accuracy   Time taken     
----------------------------------------------------------------------------------------------------
lbfgs                          0.7485                    0.7358                    0.4671         
newton-cg                      0.7488                    0.7362                    0.1401         
newton-cholesky                0.7488                    0.7362                    0.0781         
sag                            0.7488                    0.7362                    2.0829         
saga                           0.7488                    0.7362                    3.4706         


## Q4: Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

**Interpretation:** Based on the results obtained after fitting the logistic regression models with different solvers without penalties/regularizations (except liblinear which has a default l2 penalty), the model with newton-cholesky solver yielded the best results both accuracy and computation time-wise. Almost every logistic model with different solvers has the same training and holdout accuracy but with different execution times. The models with sag and saga solvers has attained the same training and holdout accuracies as of newton-cholesky but the computation times are almost 30 and 40 times that of newton-cholesky's model. Not considering the excution time, all the logistic models with different solvers have achieved the good generalizability without overfitting and underfitting and with almost the same training and holdout accuracies that is these models worked out well on the hold out subset after training. But, overall considering a combination of three parameters, training, holdout accuracies, and execution time, the model with newton-cholesky solver shows the best performance.